In [ ]:
%matplotlib inline
from math import *
import numpy as np
from matplotlib import pyplot as plt
from QuantLib import *
from PyFin.Math.Distributions import CumulativeNormalDistribution

plt.style.use('fivethirtyeight')

# 1. Functions
-------------------

In [ ]:
def bs_theoretical_price(payoff, spot, ttm, volatility, rf_rate, finance_rate=None):
    
    if not finance_rate:
        finance_rate = rf_rate
    
    forward = spot * exp(finance_rate * ttm)
    std_dev = volatility * sqrt(ttm)
    discount = exp(-rf_rate * ttm)
    return blackFormula(payoff.optionType(), payoff.strike(), forward, std_dev, discount)

In [ ]:


_dist = CumulativeNormalDistribution()

def _exercise(payoff, spot):
    return payoff(spot)

_exercise = np.frompyfunc(_exercise, 2, 1)


def _create_path(rsg, ln_spot, drift, diffusion, delta_t):
    rnd = np.array(rsg.nextSequence().value())
    in_c = delta_t * drift + np.sqrt(delta_t) * diffusion * rnd
    inc_c_cum = np.cumsum(in_c)
    return np.exp(np.concatenate(([ln_spot], ln_spot + inc_c_cum)))


def _bs_delta(option_type, finance_rate, volatility, ttm, spot, strike):
    money_ness = log(spot / strike)
    drift = (finance_rate + 0.5 * (volatility ** 2)) * ttm
    d1 = (money_ness + drift) / volatility / sqrt(ttm)
    call_delta =  _dist(d1)
    
    if option_type == Option.Call:
        return call_delta
    elif option_type == Option.Put:
        return call_delta - 1.

_bs_delta = np.frompyfunc(_bs_delta, 6, 1)


def _hedging_on_path(payoff, ttm, time_grids, spot_path, volatility, inflations, rf_rate, finance_rate, trading_cost):
    delta_t = time_grids[0] - time_grids[1]
    deltas = _bs_delta(payoff.optionType(), finance_rate, volatility, time_grids, spot_path[:-1], payoff.strike())
    borrows = spot_path[:-1] * deltas
    finance_cost = borrows * finance_rate * delta_t
    stock_pnl = deltas * (spot_path[1:] - spot_path[:-1])
    trading_slipge = np.abs(np.concatenate(([deltas[0]], np.diff(deltas)))) * spot_path[:-1] * trading_cost
    hedging_pnl = ((stock_pnl - finance_cost - trading_slipge) * inflations).sum()
    
    exercise_pnl = payoff(spot_path[-1])
    total_cost = hedging_pnl - exercise_pnl
    
    return exp(-rf_rate * ttm) * total_cost


class HedgeAnalysor(object):
    
    def __init__(self,
                 payoff,
                 trading_cost=0.):
        self.payoff = payoff
        self.trading_cost = trading_cost
    
    @staticmethod
    def _prepare_parameters(rf_rate, finance_rate, underlying_risk_return):
        if not finance_rate:
            finance_rate = rf_rate
        
        if not underlying_risk_return:
            underlying_risk_return = rf_rate
        
        return finance_rate, underlying_risk_return

    def exercise(self, spots):
        return _exercise(self.payoff, spots)
    
    def _hedge_path(self, rsg, ttm, time_grids, ln_spot, volatility, drift, diffusion, delta_t, inflations, rf_rate, finance_rate):
        spot_path = _create_path(rsg, ln_spot, drift, diffusion, delta_t)
        return _hedging_on_path(self.payoff, ttm, time_grids, spot_path, volatility, inflations, rf_rate, finance_rate, self.trading_cost)
        
    
    def hedge_cost(self,
                   rf_rate,
                   volatility,
                   realized_vol,
                   ttm,
                   finance_rate=None,
                   underlying_risk_return=None,
                   spot=1.,
                   time_steps=50,
                   simulations=100,
                   seed=20):
        rng = MersenneTwisterUniformRng(seed)
        rsg = MersenneTwisterUniformRsg(dimensionality=time_steps,
                                        rng=rng)
        rsg = InvCumulativeMersenneTwisterGaussianRsg(rsg)
        
        finance_rate, underlying_risk_return = self._prepare_parameters(rf_rate,
                                                                        finance_rate,
                                                                        underlying_risk_return)
        
        ln_spot = log(spot)
        delta_t = ttm / time_steps

        drift = underlying_risk_return - 0.5 * (realized_vol ** 2)
        diffusion = realized_vol
        time_grids = np.linspace(ttm, 0, num=time_steps, endpoint=False)
        inflations = np.exp(finance_rate * (time_grids - delta_t))
        
        hedging_cost_batch = np.zeros(simulations)
        
        for i in range(simulations):
            hedging_cost = self._hedge_path(rsg,
                                            ttm,
                                            time_grids,
                                            ln_spot,
                                            volatility,
                                            drift,
                                            diffusion,
                                            delta_t,
                                            inflations,
                                            rf_rate,
                                            finance_rate)
            hedging_cost_batch[i] = hedging_cost
        
        return -hedging_cost_batch

In [ ]:
rf_rate = 0.0435
strike = 1.
volatility = 0.20
realized_vol = 0.20
ttm = 1.
spot = 1.
trading_cost = 0.0
simulations = 20000
time_steps = 250

payoff = PlainVanillaPayoff(Option.Call, strike)
hf = HedgeAnalysor(payoff, trading_cost)

In [ ]:
%%time

res = hf.hedge_cost(rf_rate=rf_rate,
                    volatility=volatility,
                    realized_vol=realized_vol,
                    ttm=ttm, spot=spot,
                    simulations=simulations,
                    time_steps=time_steps)
bs_price = bs_theoretical_price(payoff, spot, ttm, volatility, rf_rate)

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(res, bins=50)
plt.axvline(x=bs_price, color='red', linestyle='dashed', label='Theoretical Price')
plt.axvline(x=res.mean(), color='green', linestyle='dashed', label='Hedging Cost Mean')
plt.title("Hedging Cost v.s. Theoretical Price (tc = {0})".format(trading_cost))
plt.legend()

# 2. Base Parameters
----------------------

In [ ]:
rf_rate = 0.0435
strike = 1.
ttm = 1.
spot = 1.
simulations = 20000
trading_cost = 0.002
volatility = 0.20
realized_vol = volatility

payoff = PlainVanillaPayoff(Option.Call, strike)
hf = HedgeAnalysor(payoff, trading_cost=trading_cost)

# 3. Scenario Analysis
---------------

## 2.1 Scenario 1

* `finance_rate` = `rf_rate`
* `underlying_risk_return` = `rf_rate`

In [ ]:
finance_rate = rf_rate
underlying_risk_return = rf_rate
time_steps_scenarios = [50, 250]

In [ ]:
simulations_res = []
bs_price = bs_theoretical_price(payoff, spot, ttm, volatility, rf_rate, finance_rate)

for time_steps in time_steps_scenarios:
    path_res = hf.hedge_cost(rf_rate=rf_rate,
                             volatility=volatility,
                             realized_vol=realized_vol,
                             time_steps=time_steps,
                             ttm=ttm,
                             finance_rate=finance_rate,
                             underlying_risk_return=underlying_risk_return,
                             spot=spot,
                             simulations=simulations)
    
    simulations_res.append(path_res)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

for i, ax in enumerate(axes):
    res = simulations_res[i]
    ax.hist(res, bins=50)
    ax.axvline(x=bs_price, color='red', linestyle='dashed', label='Theoretical Price')
    ax.axvline(x=res.mean(), color='green', linestyle='dashed', label='Hedging Cost Mean')
    ax.set_title("Hedging Cost v.s. Theoretical Price (tc = {0}, ts = {1})".format(trading_cost,
                                                                                   time_grids_scenarios[i]))
    ax.legend()

## 2.1 Scenario 2

* `finance_rate` = `rf_rate`
* `underlying_risk_return` = 0.12

In [ ]:
finance_rate = rf_rate
underlying_risk_return = 0.12

In [ ]:
simulations_res = []
bs_price = bs_theoretical_price(payoff, spot, ttm, volatility, rf_rate, finance_rate)

for time_steps in time_steps_scenarios:
    path_res = hf.hedge_cost(rf_rate=rf_rate,
                             volatility=volatility,
                             realized_vol=realized_vol,
                             time_steps=time_steps,
                             ttm=ttm,
                             finance_rate=finance_rate,
                             underlying_risk_return=underlying_risk_return,
                             spot=spot,
                             simulations=simulations)
    
    simulations_res.append(path_res)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

for i, ax in enumerate(axes):
    res = simulations_res[i]
    ax.hist(res, bins=50)
    ax.axvline(x=bs_price, color='red', linestyle='dashed', label='Theoretical Price')
    ax.axvline(x=res.mean(), color='green', linestyle='dashed', label='Hedging Cost Mean')
    ax.set_title("Hedging Cost v.s. Theoretical Price (tc = {0}, ts = {1})".format(trading_cost,
                                                                                   time_grids_scenarios[i]))
    ax.legend()

## 2.1 Scenario 3

* `finance_rate` = 0.10
* `underlying_risk_return` = 0.12

In [ ]:
finance_rate = 0.10
underlying_risk_return = 0.12

In [ ]:
simulations_res = []
bs_price = bs_theoretical_price(payoff, spot, ttm, volatility, rf_rate, finance_rate)

for time_steps in time_steps_scenarios:
    path_res = hf.hedge_cost(rf_rate=rf_rate,
                             volatility=volatility,
                             realized_vol=realized_vol,
                             time_steps=time_steps,
                             ttm=ttm,
                             finance_rate=finance_rate,
                             underlying_risk_return=underlying_risk_return,
                             spot=spot,
                             simulations=simulations)
    
    simulations_res.append(path_res)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

for i, ax in enumerate(axes):
    res = simulations_res[i]
    ax.hist(res, bins=50)
    ax.axvline(x=bs_price, color='red', linestyle='dashed', label='Theoretical Price')
    ax.axvline(x=res.mean(), color='green', linestyle='dashed', label='Hedging Cost Mean')
    ax.set_title("Hedging Cost v.s. Theoretical Price (tc = {0}, ts = {1})".format(trading_cost,
                                                                                   time_grids_scenarios[i]))
    ax.legend()